In [ ]:
import duckdb

# Path to your CSV file
file_path = r"C:\Users\Danyella.Santana\Downloads\Iowa_Liquor_Sales.csv"

# Create DuckDB connection
con = duckdb.connect()

# Load CSV into a virtual table (force all columns as text)
con.execute(f"""
    CREATE TABLE Iowa_Liquor_Sales AS
    SELECT * FROM read_csv_auto('{file_path}', ALL_VARCHAR=TRUE)
""")

# Preview first 5 rows
df = con.execute("SELECT * FROM Iowa_Liquor_Sales LIMIT 5").df()
print(df)

# Ver os nomes das colunas
columns = con.execute("PRAGMA table_info('Iowa_Liquor_Sales')").df()
print(columns)

In [ ]:
import pandas as pd

# Query top 50 vendors by total sales
df_vendor = con.execute(f"""
    SELECT 
        "Vendor Number", 
        "Vendor Name",
        SUM(CAST(REPLACE("Sale (Dollars)", ',', '') AS DOUBLE)) AS total_sales
    FROM read_csv_auto('{file_path}', ALL_VARCHAR=TRUE)
    GROUP BY "Vendor Number", "Vendor Name"
    ORDER BY total_sales DESC
    LIMIT 50
""").df()

# Format numbers with thousand separator and two decimal places
pd.set_option("display.float_format", "{:,.2f}".format)

print(df_vendor)



In [ ]:
import duckdb

# Path to the CSV file
file_path = r"C:\Users\Danyella.Santana\Downloads\Iowa_Liquor_Sales.csv"

# Create DuckDB connection
con = duckdb.connect()

# Load CSV into a virtual table (force all columns as text)
con.execute(f"""
    CREATE OR REPLACE TABLE Iowa_Liquor_Sales AS
    SELECT * FROM read_csv_auto('{file_path}', ALL_VARCHAR=TRUE)
""")

# Query: Top 50 Categories (with Category Number)
df_top50_categories = con.execute("""
    SELECT 
        TRY_CAST("Category" AS INT) AS category_number,
        "Category Name",
        SUM(TRY_CAST(REPLACE("Bottles Sold", ',', '') AS DOUBLE)) AS total_bottles_sold
    FROM Iowa_Liquor_Sales
    GROUP BY category_number, "Category Name"
    ORDER BY total_bottles_sold DESC
    LIMIT 50
""").df()

print(df_top50_categories)

# Note:
# I used the output of this query to identify the top 50 categories by volume.
# Then, I manually reviewed and corrected inconsistencies in the category names
# within a separate CSV document to ensure data accuracy and consistency.


In [ ]:
import duckdb

# ---------- PATHS ----------
file_path = r"C:\Users\Danyella.Santana\Downloads\Iowa_Liquor_Sales.csv"
output_items_path = r"C:\Users\Danyella.Santana\Downloads\dim_items.parquet"
output_stores_path = r"C:\Users\Danyella.Santana\Downloads\dim_stores.parquet"

# ---------- CONNECTION ----------
con = duckdb.connect()

# ---------- LOAD CSV ----------
con.execute(f"""
CREATE OR REPLACE TABLE iowa_raw AS
SELECT * FROM read_csv_auto('{file_path}', ALL_VARCHAR=TRUE)
""")

# ===============================
# DIM ITEMS - 1 row per item_number
# Criteria: highest sum of Bottles Sold
# ===============================
con.execute("""
CREATE OR REPLACE TABLE dim_items AS
WITH item_sales AS (
    SELECT
        TRY_CAST("Item Number" AS INTEGER) AS item_number,
        TRIM("Item Description") AS item_description,
        SUM(TRY_CAST("Bottles Sold" AS DOUBLE)) AS total_bottles,
        COUNT(*) AS cnt
    FROM iowa_raw
    WHERE "Item Number" IS NOT NULL
    GROUP BY "Item Number", TRIM("Item Description")
),
ranked AS (     
    SELECT
        item_number,
        item_description,
        ROW_NUMBER() OVER (
            PARTITION BY item_number
            ORDER BY total_bottles DESC, cnt DESC, item_description ASC
        ) AS rn
    FROM item_sales
)
SELECT
    CAST(item_number AS INTEGER) AS item_number,
    COALESCE(item_description, 'UNKNOWN') AS item_description
FROM ranked
WHERE rn = 1
""")

# Export Items
con.execute(f"COPY dim_items TO '{output_items_path}' (FORMAT PARQUET)")
print(" dim_items exported to", output_items_path)

# ===============================
# DIM STORES - 1 row per store_number
# store_name -> most complete (priority: Store Name > City > County)
# city/county -> most frequent value
# ===============================
con.execute("""
CREATE OR REPLACE TABLE dim_stores AS
WITH base AS (
    SELECT
        TRY_CAST("Store Number" AS INTEGER) AS store_number,
        NULLIF(TRIM("Store Name"), '') AS store_name,
        NULLIF(TRIM("City"), '') AS city,
        NULLIF(TRIM("County"), '') AS county
    FROM iowa_raw
    WHERE "Store Number" IS NOT NULL
),
-- Rule for store_name: highest completeness
scored AS (
    SELECT
        store_number,
        store_name,
        city,
        county,
        (CASE WHEN store_name IS NOT NULL THEN 1 ELSE 0 END +
         CASE WHEN city IS NOT NULL THEN 1 ELSE 0 END +
         CASE WHEN county IS NOT NULL THEN 1 ELSE 0 END) AS info_score
    FROM base
),
store_name_ranked AS (
    SELECT
        store_number,
        COALESCE(store_name, city, county, 'UNKNOWN') AS best_store_name,
        ROW_NUMBER() OVER (
            PARTITION BY store_number
            ORDER BY info_score DESC, store_name DESC
        ) AS rn
    FROM scored
),
city_mode AS (
    SELECT
        store_number,
        city,
        ROW_NUMBER() OVER (
            PARTITION BY store_number
            ORDER BY COUNT(*) DESC, city ASC
        ) AS rn
    FROM base
    WHERE city IS NOT NULL
    GROUP BY store_number, city
),
county_mode AS (
    SELECT
        store_number,
        county,
        ROW_NUMBER() OVER (
            PARTITION BY store_number
            ORDER BY COUNT(*) DESC, county ASC
        ) AS rn
    FROM base
    WHERE county IS NOT NULL
    GROUP BY store_number, county
)
SELECT
    CAST(s.store_number AS INTEGER) AS store_number,
    s.best_store_name AS store_name,
    COALESCE(c.city, 'UNKNOWN') AS city,
    COALESCE(ct.county, 'UNKNOWN') AS county
FROM store_name_ranked s
LEFT JOIN city_mode c
    ON s.store_number = c.store_number AND c.rn = 1
LEFT JOIN county_mode ct
    ON s.store_number = ct.store_number AND ct.rn = 1
WHERE s.rn = 1
""")

# Export Stores
con.execute(f"COPY dim_stores TO '{output_stores_path}' (FORMAT PARQUET)")
print(" dim_stores exported to", output_stores_path)


In [ ]:
import duckdb 
import pandas as pd

# ---------- CONFIG ----------
file_path = r"C:\Users\Danyella.Santana\Downloads\Iowa_Liquor_Sales.csv"
category_mapping_xlsx = r"C:\Users\Danyella.Santana\Downloads\Beverage_Category_Mapping_Full.xlsx"
dim_items_path = r"C:\Users\Danyella.Santana\Downloads\dim_items.parquet"
dim_stores_path = r"C:\Users\Danyella.Santana\Downloads\dim_stores.parquet"
output_path = r"C:\Users\Danyella.Santana\Downloads\Iowa_Liquor_Sales_Final.parquet"

con = duckdb.connect()

# ---------- LOAD CSV ----------
con.execute(f"""
CREATE OR REPLACE TABLE iowa_raw AS
SELECT * FROM read_csv_auto('{file_path}', ALL_VARCHAR=TRUE)
""")

# ---------- LOAD DIMENSIONS ----------
con.execute(f"CREATE OR REPLACE TABLE dim_items AS SELECT * FROM read_parquet('{dim_items_path}')")
con.execute(f"CREATE OR REPLACE TABLE dim_stores AS SELECT * FROM read_parquet('{dim_stores_path}')")

# ---------- LOAD CATEGORY MAPPING (Created manually) ----------
category_mapping_df = pd.read_excel(category_mapping_xlsx)
category_mapping_df.columns = [c.strip() for c in category_mapping_df.columns]
con.register('category_mapping', category_mapping_df)

# ---------- CLEAN AND CONVERT iowa_raw ----------
con.execute("""
CREATE OR REPLACE TABLE iowa_cleaned AS
SELECT
    EXTRACT(YEAR FROM STRPTIME("Date", '%m/%d/%Y')) AS year,
    EXTRACT(MONTH FROM STRPTIME("Date", '%m/%d/%Y')) AS month,
    CAST(EXTRACT(YEAR FROM STRPTIME("Date", '%m/%d/%Y')) AS VARCHAR) || 
    RIGHT('00' || CAST(EXTRACT(MONTH FROM STRPTIME("Date", '%m/%d/%Y')) AS VARCHAR),2) AS year_month,
    
    TRY_CAST("Store Number" AS INTEGER) AS store_number,
    COALESCE(NULLIF(TRIM("Store Name"), ''), 'Unknown') AS store_name_orig,
    COALESCE(NULLIF(TRIM("City"), ''), 'Unknown') AS city_raw,
    TRY_CAST("Vendor Number" AS INTEGER) AS vendor_number,
    COALESCE(NULLIF(TRIM("Vendor Name"), ''), 'Unknown') AS vendor_name_orig,
    TRY_CAST("Category" AS INTEGER) AS category,
    COALESCE(NULLIF(TRIM("Category Name"), ''), 'Unknown') AS category_name,
    TRY_CAST("Item Number" AS INTEGER) AS item_number,
    COALESCE(NULLIF(TRIM("Item Description"), ''), 'Unknown') AS item_description_orig,
    
    TRY_CAST("Bottle Volume (ml)" AS DOUBLE)/1000.0 AS bottle_volume_liters,
    TRY_CAST("State Bottle Retail" AS DOUBLE) AS bottle_retail,
    TRY_CAST("Bottles Sold" AS DOUBLE) AS bottles_sold,
    TRY_CAST("Sale (Dollars)" AS DOUBLE) AS sale_orig,
    TRY_CAST("Volume Sold (Liters)" AS DOUBLE) AS volume_orig,
    
    (TRY_CAST("State Bottle Retail" AS DOUBLE) * TRY_CAST("Bottles Sold" AS DOUBLE)) AS sale_calc,
    (TRY_CAST("Bottle Volume (ml)" AS DOUBLE)/1000.0 * TRY_CAST("Bottles Sold" AS DOUBLE)) AS volume_calc,
    
    1 AS sales_number
FROM iowa_raw
WHERE STRPTIME("Date", '%m/%d/%Y') BETWEEN DATE '2018-04-01' AND DATE '2023-03-31'
""")

# ---------- ENRICH WITH DIMENSIONS AND CATEGORY ---------
con.execute("""
CREATE OR REPLACE TABLE iowa_enriched AS
SELECT
    i.year,
    i.month,
    i.year_month,
    CAST(i.store_number AS VARCHAR) AS store_number,
    COALESCE(s.store_name, i.store_name_orig, 'Unknown') AS store_name,
    COALESCE(s.city, i.city_raw, 'Unknown') AS city,
    CAST(i.vendor_number AS VARCHAR) AS vendor_number,
    COALESCE(i.vendor_name_orig, 'Unknown') AS vendor_name,
    
    CASE
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('DIAGEO AMERICAS','DIAGEO','DIAGEO INC') THEN 'DIAGEO'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('SAZERAC COMPANY INC','SAZERAC COMPANY  INC','SAZERAC CO., INC','SAZERAC NORTH AMERICA','SAZERAC CO., INC.','SAZERAC COMPANY INC') THEN 'SAZERAC'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('JIM BEAM BRANDS','JIM BEAM','JIM BEAM INC') THEN 'JIM BEAM'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('PERNOD RICARD USA','PERNOD RICARD USA/AUSTIN NICHOLS') THEN 'PERNOD RICARD'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('HEAVEN HILL BRANDS','HEAVEN HILL') THEN 'HEAVEN HILL'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('FIFTH GENERATION INC','FIFTH GENERATION','FIFTH GENERATION INC.') THEN 'FIFTH GENERATION'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('PROXIMO') THEN 'PROXIMO'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('BROWN FORMAN CORP.','BROWN-FORMAN CORPORATION') THEN 'BROWN FORMAN'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('BACARDI USA INC','BACARDI U.S.A., INC.') THEN 'BACARDI'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('LUXCO INC','LUXCO-ST LOUIS') THEN 'LUXCO'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('CONSTELLATION BRANDS INC','CONSTELLATION WINE COMPANY, INC.') THEN 'CONSTELLATION BRANDS'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('MOET HENNESSY USA','MOET HENNESSY USA, INC.') THEN 'MOET HENNESSY'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('E & J GALLO WINERY','E AND J GALLO WINERY') THEN 'E & J GALLO'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('PHILLIPS BEVERAGE','PHILLIPS BEVERAGE COMPANY') THEN 'PHILLIPS BEVERAGE'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('MHW LTD') THEN 'MHW'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('REMY COINTREAU USA INC','REMY COINTREAU USA .') THEN 'REMY COINTREAU'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('LAIRD & COMPANY','LAIRD AND COMPANY') THEN 'LAIRD & COMPANY'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('MAST-JAGERMEISTER US, INC','SIDNEY FRANK IMPORTING CO.') THEN 'MAST-JAGERMEISTER'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('MCCORMICK DISTILLING CO.','MCCORMICK DISTILLING COMPANY') THEN 'MCCORMICK DISTILLING'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('INFINIUM SPIRITS') THEN 'INFINIUM SPIRITS'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('WILLIAM GRANT & SONS INC','WILLIAM GRANT AND SONS, INC.') THEN 'WILLIAM GRANT & SONS'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('PATRON SPIRITS COMPANY','THE PATRON SPIRITS COMPANY') THEN 'PATRON SPIRITS COMPANY'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('CAMPARI AMERICA','CAMPARI(SKYY)','SKYY SPIRITS INC') THEN 'CAMPARI'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('PRESTIGE WINE AND SPIRITS GROUP','PRESTIGE WINE & SPIRITS GROUP') THEN 'PRESTIGE WINE AND SPIRITS'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('STOLI GROUP') THEN 'STOLI'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('GEMINI SPIRITS') THEN 'GEMINI SPIRITS'
        WHEN UPPER(TRIM(i.vendor_name_orig)) IN ('OLE SMOKY DISTILLERY LLC') THEN 'OLE SMOKY DISTILLERY'
        ELSE 'OTHERS'
    END AS vendor_group,

    CAST(i.item_number AS VARCHAR) AS item_number,
    COALESCE(it.item_description, i.item_description_orig, 'Unknown') AS item_description,
    CAST(i.category AS VARCHAR) AS category_code,
    COALESCE(i.category_name, 'Unknown') AS category_name,
    COALESCE(cm.category_final, 'Unknown') AS category_final,
    
    i.bottles_sold,
    i.bottle_volume_liters,
    i.bottle_retail,
    i.sale_calc AS sale_dollars,
    i.volume_calc AS volume_sold_liters,
    i.sales_number,
    i.sale_orig,
    i.volume_orig
FROM iowa_cleaned i
LEFT JOIN dim_items it
    ON it.item_number = i.item_number
LEFT JOIN dim_stores s
    ON s.store_number = i.store_number
LEFT JOIN category_mapping cm
    ON LOWER(TRIM(COALESCE(i.category_name, ''))) = LOWER(TRIM(COALESCE(cm.category, '')))
""")

# ---------- FINAL FILTERS ---------
con.execute("""
DELETE FROM iowa_enriched
WHERE
    bottles_sold IS NULL OR bottles_sold <= 0
    OR bottle_retail IS NULL OR bottle_retail <= 0
    OR bottle_volume_liters IS NULL OR bottle_volume_liters <= 0
    OR sale_dollars IS NULL OR sale_dollars <= 0
    OR volume_sold_liters IS NULL OR volume_sold_liters <= 0
""")

# ---------- EXPORT FINAL ----------
con.execute(f"COPY iowa_enriched TO '{output_path}' (FORMAT PARQUET)")
print(' Exported to', output_path)
